<h1 style="color: #9966cc">Домашнее задание 1. Spark RDD API<h1>

In [2]:
import csv
import os
import sys

os.environ["SPARK_HOME"]="/home/ubuntu/BigData/spark"
os.environ["PYSPARK_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

In [3]:
import pyspark

In [4]:
conf = (
    pyspark.SparkConf()
        .set("spark.executor.memory", "1g")
        .set("spark.executor.core", "2")
        .set("spark.driver.host", "127.0.0.1")  # if you have some troubles with address binding
        .setMaster("local[2]")
)

In [5]:
sc = pyspark.SparkContext(conf=conf)
sc


<SparkContext master=local[2] appName=pyspark-shell>

<h1 style="color: #3caa3c"> Task1 <h1>

<h5 style="color: #9966cc">Вычисление расстояния от заданной точки до каждой точки общепита<h5>

In [21]:
# Local FS
FILE_PATH = "file:///home/ubuntu/BigData/datasets/places.csv"
OUTPUT_DIR_PATH = "file:///home/ubuntu/BigData/datasets/output"

In [22]:
import csv
with open("/home/ubuntu/BigData/datasets/places.csv", 'r') as f:
    reader = csv.reader(f,delimiter=",")
    list_csv=list(reader)

In [23]:
data1_rdd= sc.parallelize(list_csv)
#data1_rdd.collect()

In [24]:
def str_filter(str_list):
    return str_list[1], float(str_list[12]), float(str_list[13])

In [25]:
data1_map_rdd = data1_rdd.map(str_filter)

In [26]:
lat=55.751244 
lng=37.618423

In [27]:
from math import cos, asin, sqrt

In [28]:
def cnt_distance(str_map): #вычисление расстояния
    lat_0 = lat-str_map[2]
    lng_0 = lng-str_map[1]
    p = 0.017453292519943295
    a = 0.5 - cos((lat_0) * p)/2 + cos(lat * p) * cos(str_map[2] * p) * (1 - cos((lng_0) * p)) / 2
    c = 2 * asin(sqrt(a)) 
    r = 6371*c
    return str_map[0], float(r)

In [29]:
data2_map_rdd = data1_map_rdd.map(cnt_distance)#вычисление расстояния
data2_map_rdd.take(10)
#data2_map_rdd.collect()

[('Шоколадница', 4.481040075007144),
 ('МУ-МУ', 4.411446758630132),
 ('КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА', 4.141565403195976),
 ('Дом 12', 2.9767774781758223),
 ('Чито-Ра', 3.0348415747792776),
 ('Бар- буфет «Николай»', 2.828729205783542),
 ('Флорентини', 2.8885782352411793),
 ('Beer Gik', 1.5653137373473913),
 ('Погребок', 1.5653137373473913),
 ('Пробка Гриль', 1.6499887343355246)]

<h5 style="color: #9966cc">Вычисление расстояния между точками общепита<h5>

In [30]:
data1_map_rdd_index = data1_map_rdd.zipWithIndex()
data_cartesian_rdd = data1_map_rdd_index.cartesian(data1_map_rdd_index)\
                                        .distinct()\
                                        .filter(lambda x:x[0][1]>x[1][1])
#data_cartesian_rdd.take(1)

In [31]:
def cnt_distance_other(str_m):#вычисление расстояния
    lat0=str_m[0][0][2]
    lat1=str_m[1][0][2]
    lng0=str_m[0][0][1]
    lng1=str_m[1][0][1]
    lat_0=lat0-lat1
    lng_0 = lng0-lng1
    p = 0.017453292519943295
    a = 0.5 - cos((lat_0) * p)/2 + cos(lat0 * p) * cos(lat1 * p) * (1 - cos((lng_0) * p)) / 2
    c = 2 * asin(sqrt(a)) 
    r = 6371*c
    return str_m[0][0][0],str_m[1][0][0], float(r)

In [32]:
data4_map_rdd = data_cartesian_rdd.map(cnt_distance_other)#вычисление расстояния
data2_map_rdd.take(10)
#data4_map_rdd.collect()

[('Шоколадница', 4.481040075007144),
 ('МУ-МУ', 4.411446758630132),
 ('КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА', 4.141565403195976),
 ('Дом 12', 2.9767774781758223),
 ('Чито-Ра', 3.0348415747792776),
 ('Бар- буфет «Николай»', 2.828729205783542),
 ('Флорентини', 2.8885782352411793),
 ('Beer Gik', 1.5653137373473913),
 ('Погребок', 1.5653137373473913),
 ('Пробка Гриль', 1.6499887343355246)]

<h5 style="color: #9966cc">Вывод 10 наиболее близких заведений<h5>

In [33]:
place_bliz=data4_map_rdd.sortBy(lambda x:x[2])
place_bliz.take(10)

[('Фуршет.ру', 'КАФЕДРА ПИТАНИЯ ПРИ МИГАИКЕ', 0.0),
 ('Kozlovna', 'Beer Gik', 0.0),
 ('PIZZETTA', 'Pizzetta', 0.0),
 ('Кофейня «Правда кофе»', 'Pizzetta', 0.0),
 ('Кофейня «Правда кофе»', 'PIZZETTA', 0.0),
 ('Beer Harbor', 'Кафе Кальян', 0.0),
 ('Коленки Пчелы', 'Блэк милк', 0.0),
 ('Мегобари', 'Jeffreys coffee', 0.0),
 ('Грузины Венахи готовят', 'Тайская кухня «Том-Ям»', 0.0),
 ('Табу бар вайн', 'Грузины Венахи готовят', 0.0)]

<h5 style="color: #9966cc">Вывод 10 наиболее отдаленных заведений<h5>

In [34]:
place_dal=data4_map_rdd.sortBy(lambda x:x[2], ascending=False)
place_dal.take(10)

[('Политех', 'МНИТИ', 5.456078051682729),
 ('Залечь на дно', 'МНИТИ', 5.456078051682729),
 ('Антикафе Checkpoint', 'МНИТИ', 5.456078051682729),
 ('Calabash Club', 'МНИТИ', 5.456078051682729),
 ('Шоколадница', 'МНИТИ', 5.402045734207267),
 ('Мареа', 'МНИТИ', 5.397580177699907),
 ('БИБЛИОТЕКА Shisha Lounge', 'МНИТИ', 5.394562231716176),
 ('Му-Му', 'МНИТИ', 5.394562231716176),
 ('Стейк Хаус «Бизон»', 'МНИТИ', 5.393724300516158),
 ('Jimmy Poy', 'МНИТИ', 5.3820992042757405)]

<h3> Task2 <h3>

In [35]:
import json 

In [36]:
FILE_PATH = "file:///home/ubuntu/BigData/datasets/Electronics_5.json"
OUTPUT_DIR_PATH = "file:///home/ubuntu/BigData/datasets/output"

<h5 style="color: #9966cc">Расчет среднего рейтинга товаров<h5>

In [38]:
json_reviews_rdd = sc.textFile(FILE_PATH)
def json_parse(file):
    result=json.loads(file)
    return result["asin"],float(result["overall"])
data_rdd=json_reviews_rdd.map(json_parse)
#data_rdd.take(10)

In [39]:
avg_prod_rating_rdd = data_rdd \
    .aggregateByKey((0,0), 
                    lambda x, value: (x[0] + value, x[1] + 1), 
                    lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .mapValues(lambda x: x[0]/x[1])

avg_prod_rating_rdd.take(10)

[('B00000J4GE', 4.25),
 ('B00001OPJV', 3.2),
 ('B00004RDMR', 4.470588235294118),
 ('B00004TEN2', 4.071428571428571),
 ('B00004TY7Q', 4.6),
 ('B00004U4MT', 4.363636363636363),
 ('B00004U8K4', 1.8),
 ('B00004Z5M1', 4.681159420289855),
 ('B00005219A', 4.722222222222222),
 ('B00005LD4T', 4.545454545454546)]

<h5 style="color: #9966cc">Сопоставление среднего рейтинга товаров с их наименованием<h5>

In [40]:
# Local FS
FILE_PATH = "file:///home/ubuntu/BigData/datasets/meta_Electronics.json"
OUTPUT_DIR_PATH = "file:///home/ubuntu/BigData/datasets/output"

In [41]:
json_reviews_rdd_title = sc.textFile(FILE_PATH)
def json_parser_name(file):
    try:
        result_name = eval(file)
        return result_name["asin"], result_name["title"]
    except:
        return result_name["asin"], "unknown film"
data_rdd_title_filt=json_reviews_rdd_title.map(json_parser_name)\
                    .filter(lambda x:x!=1 )
data_rdd_title_filt.take(10)

[('0132793040',
  'Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker'),
 ('0321732944',
  'Kelby Training DVD: Adobe Photoshop CS5 Crash Course By Matt Kloskowski'),
 ('0439886341', 'Digital Organizer and Messenger'),
 ('0511189877', 'CLIKR-5 Time Warner Cable Remote Control UR5U-8780L'),
 ('0528881469',
  'Rand McNally 528881469 7-inch Intelliroute TND 700 Truck GPS'),
 ('0558835155',
  'Polaroid Pbm2200 PC / Gaming Stereo Headphones With Microphone &amp; In-line Volume'),
 ('0594012015', 'Barnes &amp; Noble NOOK Power Kit in Carbon BNADPN31'),
 ('0594017580', 'Jonathan Adler Punctuation Cover in Black'),
 ('0594450209', 'Barnes &amp; Noble 7&quot; Lautner Chocolate NOOK Case'),
 ('0594287995',
  'Kate Spade Rain or Shine Magazine Cover for Nook Simple Touch')]

In [42]:
prod_name =avg_prod_rating_rdd.join(data_rdd_title_filt)
#prod_name.take(2)

[('B0000CO2UT',
  (2.4285714285714284,
   'Harman Kardon FL8385 5-Disc CD Changer (Discontinued by Manufacturer)')),
 ('B0000EI9VB',
  (3.6666666666666665, 'Kodak K5000-C Li-Ion Rapid Battery Charger Kit'))]

<h5 style="color: #9966cc">RDD с рейтингом товаров меньше 3<h5>

In [43]:
rating_less_three=prod_name.filter(lambda x: x[1][0]<3)\
                  .sortBy(lambda x: x[1][0])
rating_less_three.take(10)

[('B001BLN91Y',
  (1.0,
   'Maxtor OneTouch 4 - 1 TB USB 2.0 Desktop External Hard Drive STM310005OTA3E1-RK')),
 ('B003KIQTXG',
  (1.0,
   'NEEWER&reg; Photographic Barn Door &amp; Honeycomb Grid &amp; Gel Set for Alienbees Alienbee Flash')),
 ('B00111JODG',
  (1.0,
   'StarTech HDMISPL1HH 1 feet Standard HDMI Cable - 1x HDMI (M) to 2x HDMI (F) (Discontinued by Manufacturer)')),
 ('B000NNFS4C',
  (1.0, 'RCA DRC8335 DVD Recorder &amp; VCR Combo With Built-In Tuner')),
 ('B000H13L4Y', (1.0, 'ATI TV Wonder 200 PCI Video Card w/PVR Capabilities')),
 ('B000F1ORW6', (1.0, 'GE 24746 Futura HDTV Ready Antenna')),
 ('B001T9N0R6',
  (1.0, 'Zeikos 57-in-1 USB 2.0 Flash Memory Card Reader ZE-CR201')),
 ('B00000JBIA', (1.0, 'Agfa ePhoto SMILE 0.2MP Digital Camera')),
 ('B0013WI5SS', (1.0, 'Dynex-DX-AP100 Adapter Mini DVI to Mini-DIN')),
 ('B004K9CL6S',
  (1.0,
   'NEW SYLVANIA HD1Z SDSDHCMMC 720P HARD DRIVE POCKET VIDEO DIGITAL CAMERACAMCORDER W4X DIGITAL ZOOM HDMI &amp; 2 LCD PEACOCK BLUE'))]

<h5 style="color: #9966cc">Сохранение RDD в файл<h5>

In [46]:
rating_less_three.coalesce(1)\
                 .saveAsTextFile(OUTPUT_DIR_PATH)

<h1> Task3 <h1>

<h5 style="color: #9966cc"> Вычисление косинусного сходства между фильмами<h5>

In [6]:
# Local FS
FILE_PATH = "file:///home/ubuntu/datasets/ratings.csv"
OUTPUT_DIR_PATH = "file:///home/ubuntu/BigData/datasets/output"

In [7]:
rat_films_rdd = sc.textFile(FILE_PATH)

In [8]:
def csv_parse(file_csv):
    result = file_csv.split(",")
    try:
        return result[0], result[1], float(result[2])
    except:
        return 1
data_rat_film=rat_films_rdd.map(csv_parse)\
                           .filter(lambda x: x!=1)\
                           .sample(False, 0.6, seed=1)\
                           .repartition(10)\
                           .zipWithIndex()
#data_rat_film.take(10)

[(('1', '2617', 2.0), 0),
 (('1', '2628', 4.0), 1),
 (('1', '2641', 5.0), 2),
 (('1', '2644', 4.0), 3),
 (('1', '2654', 5.0), 4),
 (('1', '2761', 5.0), 5),
 (('1', '2797', 4.0), 6),
 (('1', '2826', 4.0), 7),
 (('1', '2858', 5.0), 8),
 (('1', '2899', 5.0), 9)]

In [9]:
data_film_cart= data_rat_film.cartesian(data_rat_film)\
                             .filter(lambda x:x[0][1]<x[1][1])\
                             .filter(lambda x:x[0][0][0]==x[1][0][0])
#data_film_cart.take(5)

[((('1', '2617', 2.0), 0), (('1', '2628', 4.0), 1)),
 ((('1', '2617', 2.0), 0), (('1', '2641', 5.0), 2)),
 ((('1', '2617', 2.0), 0), (('1', '2644', 4.0), 3)),
 ((('1', '2617', 2.0), 0), (('1', '2654', 5.0), 4)),
 ((('1', '2617', 2.0), 0), (('1', '2761', 5.0), 5))]

In [10]:
from math import sqrt
data_film = data_film_cart.map(lambda x: ((x[0][0][1], x[1][0][1]), x[0][0][2]*x[1][0][2]))       
#data_film.take(5)
data_film_chisl=data_film.reduceByKey(lambda x, value: x+value)
#data_film_chisl.take(5)

[(('2644', '2858'), 20.0),
 (('319', '4239'), 35.5),
 (('357', '4902'), 38.0),
 (('7451', '8636'), 108.5),
 (('39', '261'), 138.5)]

In [46]:
from math import sqrt
data_rat=data_rat_film.map(lambda x: (x[0][1],x[0][2]**2))

data_film_abs=data_rat.reduceByKey(lambda x,value: x+value)\
                      .map(lambda x: (x[0], sqrt(x[1])))\
                      .zipWithIndex()
data_film_abs_znam = data_film_abs.cartesian(data_film_abs)\
                                  .filter(lambda x:x[0][1]<x[1][1])\
                                  .filter(lambda x:x[0][0][0]!=x[1][0][0])\
                                  .map(lambda x: ((x[0][0][0],x[1][0][0]),(x[0][0][1]*x[1][0][1])))
#data_film_abs_znam.take(5)

[(('319', '553'), 375.25624578413084),
 (('319', '248'), 175.43695448792994),
 (('319', '637'), 203.2993482527674),
 (('319', '667'), 99.53705340223812),
 (('319', '697'), 82.8198345808539)]

In [47]:
data_film_cos = data_film_chisl.join(data_film_abs_znam)
data_film_cos_res = data_film_cos.map(lambda x: (x[0],x[1][0]/x[1][1]))

#data_film_cos_res.take(5)

[(('2644', '2858'), 0.04786469966576814),
 (('1077', '3623'), 0.043200823821115744),
 (('1079', '2502'), 0.10984090372009993),
 (('1924', '3584'), 0.17118419700436519),
 (('40815', '122882'), 0.17949130790100784)]

In [48]:
film_cos=data_film_cos_res.map(lambda x: (x[0][0], x[0][1], x[1]))
film_cos.take(5)

[('2644', '2858', 0.04786469966576814),
 ('1077', '3623', 0.043200823821115744),
 ('1079', '2502', 0.10984090372009993),
 ('1924', '3584', 0.17118419700436519),
 ('40815', '122882', 0.17949130790100784)]

<h5 style="color: #9966cc">RDD для movieId равным 589<h5> 

In [49]:
film_cos_589= film_cos.filter(lambda x: x[0]=="589" or x[1]=="589")
film_cos_589.take(5)

[('7153', '589', 0.07603093432293428),
 ('6217', '589', 0.07080707990591183),
 ('5377', '589', 0.047297558608447206),
 ('3752', '589', 0.09080061344590405),
 ('7169', '589', 0.0515045719694647)]

<h5 style="color: #9966cc"> Добавление наименования похожим фильмам<h5>

In [50]:
def name_film(res):
    if (int(res[0])==589):
        return res[1], (res[0], res[2])
    if (int(res[1])==589):
        return res[0], (res[1], res[2])
rdd_id_film = film_cos_589.map(name_film)
#rdd_id_film.take(5)

[('7153', ('589', 0.07603093432293428)),
 ('6217', ('589', 0.07080707990591183)),
 ('5377', ('589', 0.047297558608447206)),
 ('3752', ('589', 0.09080061344590405)),
 ('7169', ('589', 0.0515045719694647))]

In [51]:
# Local FS
FILE_PATH = "file:///home/ubuntu/datasets/movies.csv"
OUTPUT_DIR_PATH = "file:///home/ubuntu/BigData/datasets/output"

In [52]:
name_films_rdd = sc.textFile(FILE_PATH)

In [53]:
def csv_parse_name(file_csv):
    result = file_csv.split(",")
    try:
        return result[0], result[1]
    except:
        return 1
name_film=name_films_rdd.map(csv_parse_name)\
                           .filter(lambda x: x!=1)\
                            .filter(lambda x: x[0]!="movieId")
#name_film.take(10)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)'),
 ('4', 'Waiting to Exhale (1995)'),
 ('5', 'Father of the Bride Part II (1995)'),
 ('6', 'Heat (1995)'),
 ('7', 'Sabrina (1995)'),
 ('8', 'Tom and Huck (1995)'),
 ('9', 'Sudden Death (1995)'),
 ('10', 'GoldenEye (1995)')]

In [54]:
name_fimiliar_film=rdd_id_film.join(name_film).map(lambda x: (x[1][0][0], x[0], x[1][1], x[1][0][1]))
#name_fimiliar_film.take(10)

[('589', '74075', 'Stan Helsing (2009)', 0.06785547598194783),
 ('589', '4519', '"Land Before Time', 0.07802670566915001),
 ('589', '7069', '"Macbeth (a.k.a. Tragedy of Macbeth', 0.06419454749779736),
 ('589',
  '66',
  'Lawnmower Man 2: Beyond Cyberspace (1996)',
  0.020653098453432182),
 ('589', '5456', 'Wagons East (1994)', 0.08529967503551326),
 ('589', '2745', '"Mission', 0.08793468750895637),
 ('589', '3156', 'Bicentennial Man (1999)', 0.13977068129614895),
 ('589', '3412', '"Bear', 0.04482385219291597),
 ('589', '38798', 'In Her Shoes (2005)', 0.029882568128610645),
 ('589', '36397', 'Valiant (2005)', 0.08529967503551328)]

<h5 style="color: #9966cc">Вывод 10 наиболее похожих фильмов<h5>

In [55]:
rat_film_sort=name_fimiliar_film.sortBy(lambda x: x[3], ascending = False)
rat_film_sort.take(10)

[('589', '480', 'Jurassic Park (1993)', 0.4305079793627106),
 ('589', '592', 'Batman (1989)', 0.41754823116964357),
 ('589', '380', 'True Lies (1994)', 0.3650254594508409),
 ('589', '457', '"Fugitive', 0.3646481252265604),
 ('589', '377', 'Speed (1994)', 0.3633615843957008),
 ('589',
  '32',
  'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
  0.33684139691551196),
 ('589', '110', 'Braveheart (1995)', 0.33237183937924),
 ('589', '296', 'Pulp Fiction (1994)', 0.3294827352039905),
 ('589', '367', '"Mask', 0.32313253168594863),
 ('589', '587', 'Ghost (1990)', 0.3111574823207267)]

In [ ]:
# sc.stop()